In [1]:
import math
import random
import pandas as pd
import numpy as np
import os

from typing import List, TypedDict
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from openai import OpenAI

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)

## Загрузка данных

In [ ]:
data = pd.read_json("data_final_for_dls_new.jsonl", lines=True)
data.columns = ['text', 'address', 'name', 'norm_name_ru', 'permalink', 'prices_summarized', 'relevance', 'reviews_summarized', 'relevance_new']
train_data = data[570:]
eval_data = data[:570]
train_data = train_data[train_data["relevance"] != 0.1].reset_index(drop=True)
eval_data = eval_data[eval_data["relevance"] != 0.1].reset_index(drop=True)

Проводилась небольшая подготовка данных. В колонку summarized объединялись данные из 'norm_name_ru', 'prices_summarized', 'reviews_summarized'. "Лишние" колонки затем удалялись, для более удобного восприятия и отображения датасета.

In [ ]:
eval_data['reviews_summarized'] = eval_data['reviews_summarized'].fillna(";")
eval_data['prices_summarized'] = eval_data['prices_summarized'].fillna(";")
eval_data['reviews_summarized'] = eval_data['reviews_summarized'].str.split(r'[\n|]').str[0].str.strip()
eval_data['summarized'] = (eval_data['norm_name_ru'] + ' ; ' + eval_data['prices_summarized'] + ' ; ' + eval_data['reviews_summarized'])
eval_data.drop(columns=['norm_name_ru', 'prices_summarized', 'reviews_summarized', 'permalink'], inplace=True)

In [ ]:
# eval_data.to_excel('eval_data_summarized.xlsx')

In [3]:
eval_data = pd.read_excel('eval_data_summarized.xlsx')
eval_data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
eval_data.head(2)

,text,address,name,relevance,relevance_new,summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,1,1,Магазин табака и курительных принадлежностей ;...
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,0,0,Кафе ; PioNero предлагает разнообразные блюда ...


In [6]:
eval_data.shape

(500, 6)

### gpt-4.1-mini - финальный промпт для классификации

In [7]:
# Вызываем модель и передаем ей инструменты
load_dotenv()
client = OpenAI(
    base_url="https://api.ai-mediator.ru/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [8]:
# Определение типов для структуры данных
class OrganizationData(TypedDict):
    user_query: str
    organization_address: str
    organization_name: str
    summary: str
    classification: str

Тут представлена версия промпта, доработанная в ходе построения агента. Это финальная версия для классификационного этапа.

In [9]:
# Системный промпт для классификации

SYSTEM_PROMPT = """Ты помощник для классификации организаций по релевантности пользовательскому запросу.
Оцени, подходит ли организация к данному запросу. Отвечай только "relevant" или "irrelevant".

**Правила:**
1. Сначала определи, есть ли в запросе географические указания (город, район, адрес). Если нет - пункт 2 не применяй.
2. Учитывай географическую принадлежность к городу (ТОЛЬКО если в запросе явно указан город)
3. Учитывай совпадение по ключевым словам ("налоговая", "больница", "ресторан" и т.д.):
   - анализируй тип деятельности организации и соответствие запросу — даже если совпадений по словам немного, 
   но смысловой контекст пересекается, отмечай как relevant.
4. Учитывай совпадение по номеру (если цифры есть в запросе)
5. Оцени другие возможные критерии, исходя из которых можно сделать вывод "relevant" или "irrelevant".

**Примеры:**
Запрос "налоговая 5007" → проверяй только ключевые слова и номер
Запрос "больница в Королёве" → проверяй ключевые слова и геолокацию
"""

**в `classify_organization` прописать модель**

In [10]:
def classify_organization(row: pd.Series) -> str:
    """Определяет релевантность организации запросу пользователя"""
    
    # Формируем пользовательский промпт
    user_prompt = f"""
    ЗАПРОС ПОЛЬЗОВАТЕЛЯ: {row['text']}
    
    ИНФОРМАЦИЯ ОБ ОРГАНИЗАЦИИ:
    - Адрес: {row['address']}
    - Название: {row['name']}
    - Сводка: {row['summarized']}
    """
    
    # Вызов модели
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    
    # Получаем и возвращаем классификацию
    return response.choices[0].message.content.strip().lower()

In [11]:
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Обрабатывает DataFrame с данными организаций"""
    
    # Создаем копию DataFrame для результатов
    result_df = df.copy()
    
    # Добавляем столбец с классификацией
    tqdm.pandas(desc="Classifying organizations")
    result_df['llm_rel'] = df.progress_apply(classify_organization, axis=1)
    
    return result_df

In [12]:
# Обрабатываем данные
classified_data = process_dataframe(eval_data)

Classifying organizations:   0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
classified_data['llm_rel'] = np.where(classified_data['llm_rel'].str.contains('irrelevant'), 0, 1)

In [14]:
classified_data

,text,address,name,relevance,relevance_new,summarized,llm_rel
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,1,1,Магазин табака и курительных принадлежностей ;...,1
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,0,0,Кафе ; PioNero предлагает разнообразные блюда ...,0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,1,1,Стоматологическая клиника ; Стоматологическая ...,1
3,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,1,1,Магазин бытовой техники ; М.Видео предлагает ш...,1
4,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,1,1,Быстрое питание ; Rostic's предлагает различны...,1
...,...,...,...,...,...,...,...
495,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,1,1,Салон красоты ; Салон красоты «Сила» предлагае...,1
496,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,0,0,Компьютерный клуб ; YouPlay КиберКлуб предлага...,1
497,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,0,0,Телекоммуникационная компания ; ; ; ;,1
498,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,0,0,"База , дом отдыха ; Предлагает размещение в ра...",1


In [15]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(classified_data['relevance'], classified_data['llm_rel'])
accuracy_new = accuracy_score(classified_data['relevance_new'], classified_data['llm_rel'])
print(f"Accuracy: {accuracy:.2f}", f"Accuracy_new: {accuracy_new:.2f}", sep='\n')

Accuracy: 0.66
Accuracy_new: 0.77


In [16]:
classified_data.to_excel('4_1mini_test_final_prompt.xlsx')

**Вывод**  
Для данной модели финальная версия классификационного промпта позволила увеличить метрику до 77% на новой разметке. Это лишь на 1% выше относительно стартовой версии промпта.  
Это подтверждает более высокую стабильность данной модели LLM.